In [ ]:
# Cell 1 — Imports, Tracing & Environment

# Standard boilerplate — same opening as every agents notebook

# Identical to Notebook 02's Cell 1. This is the standard opening block for every notebook that uses the Agents SDK — tracing off, .env loaded, OpenRouter configured, model set.

import os
from dotenv import load_dotenv
from agents import Agent, Runner, set_tracing_disabled

# Disable tracing FIRST — silences 401 errors from OpenRouter usage
set_tracing_disabled(True)

load_dotenv(override=True)

os.environ["OPENAI_API_KEY"]  = os.getenv("OPENROUTER_API_KEY")
os.environ["OPENAI_BASE_URL"] = "https://openrouter.ai/api/v1"

MODEL = os.getenv("MODEL", "openai/gpt-4o-mini")

print("✅ Environment ready. Tracing disabled.")
print(f"   Model: {MODEL}")

In [2]:
# Cell 2 — Define the Test Snippet

# A class with multiple methods — richer than Notebook 02

# A more complex snippet this time — a class with multiple methods. This gives the Documentation Writer richer content to work with and shows how the pipeline handles real-world code structure.

test_code = """
class UserValidator:
    def __init__(self, min_age: int = 18, allowed_domains: list = None):
        self.min_age = min_age
        self.allowed_domains = allowed_domains or []

    def validate_age(self, age: int) -> bool:
        return age >= self.min_age

    def validate_email(self, email: str) -> bool:
        if "@" not in email:
            return False
        domain = email.split("@")[1]
        if self.allowed_domains:
            return domain in self.allowed_domains
        return True

    def validate_user(self, age: int, email: str) -> dict:
        return {
            "valid": self.validate_age(age) and self.validate_email(email),
            "age_valid": self.validate_age(age),
            "email_valid": self.validate_email(email)
        }
"""

print("Test code ready.")

Test code ready.


In [3]:
# Cell 3 — Create Both Agents

# Code Analyst + Documentation Writer — two focused system prompts

# Notice how the Documentation Writer's system prompt is written specifically for its position in the pipeline — it knows it receives structured analysis as input, not raw code. Each agent's prompt is tuned for exactly what it receives and exactly what it must produce. This is what makes multi-agent pipelines outperform single-agent approaches.

# ── Agent 1: Code Analyst ─────────────────────────────────────────────────
code_analyst = Agent(
    name="Code Analyst",
    instructions="""
    You are an expert code analyst. Read the provided Python or JavaScript
    code and produce a thorough structured analysis.

    For every function, method, and class identify:
    - Name and purpose (plain English, 1-2 sentences)
    - All parameters: names, types, what they represent
    - Return value: type and what it contains
    - Edge cases, error conditions, and assumptions
    - Dependencies between functions and methods

    Be precise and complete. Your output goes directly to a documentation
    writer agent — accuracy is critical.
    """,
    model=MODEL
)

# ── Agent 2: Documentation Writer ────────────────────────────────────────
# This agent knows it receives STRUCTURED ANALYSIS — not raw code.
# Its prompt is written for its position in the pipeline.
doc_writer = Agent(
    name="Documentation Writer",
    instructions="""
    You are a professional technical documentation writer.
    You will receive a structured analysis of Python or JavaScript code.

    Using that analysis, produce complete professional documentation
    in Markdown format including:

    ## Overview
    Plain-English summary of what this code does and what problem it solves.

    ## Classes / Functions
    For each: docstring-style description, parameters table
    (name | type | description), return value, raises if applicable.

    ## Usage Examples
    At least 2 realistic, runnable code examples.

    ## Edge Cases & Warnings
    Important limitations, gotchas, or error conditions.

    Write for developers. Be clear, complete, professional.
    Use proper Markdown formatting throughout.
    """,
    model=MODEL
)

print("✅ Both agents created")
print(f"   Agent 1: {code_analyst.name}")
print(f"   Agent 2: {doc_writer.name}")

✅ Both agents created
   Agent 1: Code Analyst
   Agent 2: Documentation Writer


In [4]:
# Cell 4 — Run the Two-Agent Pipeline

# Run Agent 1, then hand its output to Agent 2

# This cell contains the most important pattern in the entire project. Agent 1 runs on raw code. Its output — result_1.final_output — becomes the input to Agent 2. The writer never sees the raw code. This is the handoff.

# The handoff in one line:
# result_2 = await Runner.run(agent_2, result_1.final_output)
# ── Step 1: Run the Code Analyst ─────────────────────────────────────────
print("Step 1: Running Code Analyst...")
print("-" * 60)

# await Runner.run() — correct for Jupyter. Never run_sync() in notebooks.
result_1 = await Runner.run(
    code_analyst,
    f"Analyse this code:\n\n{test_code}"
)

analysis_text = result_1.final_output
print("Code Analyst output:")
print(analysis_text)
print()

# ── Step 2: THE HANDOFF ───────────────────────────────────────────────────
# analysis_text (Agent 1's output) becomes Agent 2's input.
# The writer never sees raw code — only the analyst's structured extraction.
print("Step 2: Running Documentation Writer...")
print("-" * 60)

result_2 = await Runner.run(
    doc_writer,
    f"""Using this structured analysis, write complete
professional documentation:

{analysis_text}

Original code for reference:
{test_code}"""
)

documentation = result_2.final_output
print("Documentation Writer output:")
print(documentation)

Step 1: Running Code Analyst...
------------------------------------------------------------
Code Analyst output:
# Analysis of the `UserValidator` Class

## Class: `UserValidator`

### Purpose
The `UserValidator` class is designed to validate user input, specifically checking if a user meets the minimum age requirement and if their email belongs to an accepted domain.

### Attributes
- `min_age`: (int) Represents the minimum age required for validation.
- `allowed_domains`: (list) A list of allowed email domains that a user's email must belong to for the validation to be successful.

### Constructor
#### `__init__(self, min_age: int = 18, allowed_domains: list = None)`

- **Purpose**: Initializes the `UserValidator` instance with default parameters for minimum age and allowed email domains.
- **Parameters**:
  - `min_age` (int): The minimum age for validation; defaults to 18 if not provided.
  - `allowed_domains` (list): A list of allowed email domains; defaults to an empty list if no

In [5]:
# Cell 5 — Compare Input vs Output Sizes

# Sense-check — docs should be much richer than raw code

print("Pipeline statistics:")
print(f"  Original code:        {len(test_code):>6} characters")
print(f"  Analysis output:      {len(analysis_text):>6} characters")
print(f"  Documentation output: {len(documentation):>6} characters")

ratio = round(len(documentation) / max(len(test_code), 1), 1)
print(f"\n  Documentation is ~{ratio}x longer than the original code.")



# The handoff pattern — memorise this
# # Step 1: Run first agent on raw input
# result_1 = await Runner.run(agent_1, raw_code)

# # Step 2: Agent 1's output becomes Agent 2's input
# result_2 = await Runner.run(agent_2, result_1.final_output)

# # Step 3: Agent 2's output becomes Agent 3's input
# result_3 = await Runner.run(agent_3, result_2.final_output)

Pipeline statistics:
  Original code:           772 characters
  Analysis output:        3874 characters
  Documentation output:   4924 characters

  Documentation is ~6.4x longer than the original code.


In [ ]:
# ✅ Notebook 03 Complete
# Built and ran a two-agent sequential pipeline
# Agent 1 output handed off as Agent 2 input
# Understood why focused agents outperform single-agent approaches
# The handoff pattern is exactly what the final app uses with 3 agents
# → Next: 04_markdown_file_generation.ipynb